In [2]:
from IPython.display import display, HTML
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', 6)

In [3]:
# pandas-datareader package has a DataReader class for Yahoo finance, Google finance, etc

import pandas_datareader.data as web
# from datetime import datetime

df = pd.DataFrame()
symbols = 'GOOG IBM GE TSLA AAPL'.split()
for symbol in symbols:
    df_onesym = web.DataReader(symbol, 'yahoo', '2010-07-17', '2017-10-08')
    for name in df_onesym:
        newname = symbol.lower() + '_' + name.replace(' ', '_').lower()
        df[newname] = df_onesym[name]
df.to_csv('../shared-resources/time_series/time-series.csv')
df = df.fillna(method='ffill')
df = df.reindex()

RemoteDataError: Unable to read URL: https://query1.finance.yahoo.com/v7/finance/download/GOOG?period1=1279350000&period2=1507532399&interval=1d&events=history&crumb=A%5Cu002FsDJNoFYj1

In [4]:
type(df.index)  # dtype object

pandas.core.indexes.base.Index

In [5]:
df.index.name = df.index.name.lower().replace(' ', '_')
df.index = pd.to_datetime(df.index)
df = df.fillna(method='ffill')
df = df.reindex()
df.to_csv('../shared-resources/time_series/time-series.csv')

AttributeError: 'NoneType' object has no attribute 'lower'

In [6]:
df

""


In [7]:
df.describe()

ValueError: Cannot describe a DataFrame without columns

In [8]:
df = pd.read_csv('../shared-resources/time_series/time-series.csv', index_col=0, header=0, parse_dates=True)
type(df.index)
df.index.isnull().sum()

0

In [9]:
df = pd.read_csv('../shared-resources/time_series/time-series.csv', index_col=0, header=0, parse_dates=True)
btc = pd.read_csv('../shared-resources/time_series/coindesk-bpi-USD-ohlc_data-2010-06-30_2017-10-08.csv',
                  index_col=0, header=0, parse_dates=True)
btc.head()
#btc.index = btc.index.astype(df.index.dtype)
# for col in btc.columns:
#     df[col.replace(' ', '_').lower()] = btc[col]
# df.to_csv('../shared-resources/time_series/time-series2.csv')
# df.head()

,Open,High,Low,Close
Date,,,,
2010-07-18,0.05,0.09,0.06,0.09
2010-07-19,0.09,0.09,0.08,0.08
2010-07-20,0.08,0.08,0.07,0.07
2010-07-21,0.07,0.08,0.07,0.08
2010-07-22,0.08,0.08,0.05,0.05


In [10]:
# transform dataframe
btc['rise'] = btc.Close.diff()
btc_cleaned = btc.iloc[1:]
btc_cleaned.head()

,Open,High,Low,Close,rise
Date,,,,,
2010-07-19,0.09,0.09,0.08,0.08,-0.01
2010-07-20,0.08,0.08,0.07,0.07,-0.01
2010-07-21,0.07,0.08,0.07,0.08,0.01
2010-07-22,0.08,0.08,0.05,0.05,-0.03
2010-07-23,0.05,0.07,0.05,0.06,0.01


In [11]:
#do forestregression and things
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators=10, max_depth=2, n_jobs=-1)

X = (btc_cleaned.Close[:-5],btc_cleaned.Close[:-4])
y = btc_cleaned.rise[5:]
# try to compare to previous values and compare rise and drop with previous day
reg.fit(X,y)
reg.score(X,y)
btc_cleaned['rise_pred'] = reg.predict(btc_cleaned.rise)


ValueError: setting an array element with a sequence.

In [12]:
import numpy as np
import matplotlib
import seaborn as sns

In [13]:
ax = btc_cleaned[["rise", "rise_pred"]].plot(figsize=(16, 6))
plt.show()

KeyError: "['rise_pred'] not in index"

## Your Turn

1. load the time series of bitcoin prices from the ../shared-resources/ directory
2. make sure there aren't any "Unknown" columns of integers in your
2. make sure the index has an appropriate type for a *time series*
3. add a column to this time-series.csv file for bitcoin
3. save it to 'time-series-with-bitcoin.csv'

## BONUS

1. find a stock/commodity/index you're interested in: '../shared-resources/time_series/secwiki_tickers.csv'
2. use yahoo or google finance to download it's data
3. Add columns for it to your time series DataFrame
4. save it to a csv for use later in class